# Research Notebook
## Zachary Doucet
## Date: 10 March 2025

# 1: Experience
## Describe at least one research activity you worked on this week. 

Since the last research journal I have continued working on my code for the metric function g00 for the interior of a dark star and trying to get it to match up with the Schwarzschild solution for outside the dark star.


## Motivation:

The motivation behind this activity is to be able to determine the wavelength of light from an acreting dark star. Computing this metric function will also allow me to get the potential energy as a function of the radius from the centre of the star and the Z redshift factor. Once combined with my peers individual projects, we will be able to determine the wavelength.

# 2: What? (What happened?)

For this activity, I have already solved for the metric function g00 as per my last research journal. I built on this code and have researched and attempted to implement the Schwarzschild solution [1]. This portion was easy enough as it is simply $1 - \frac{2GM}{R}$. The difficulty however has come from rescaling my metric function such that these two solution match up at the edge of the star. In the process of trying to rescale my metric function I noticed some errors such as an incorrect value of the gravitational constant G, where I was meant to have this on the order of $10^19$ however I had $10^-19$. Additionally, Dr. Schaffner-Bielich recommended I start at a larger initial radius than the $10^{-7}$. after correcting these error, my differential equation solution diverges. I thus have shifted my focus to trying to get the solution for the metric function to converge (ie exactly what was in my last research journal). 

Recall that the equations of interest are as follows:

$\frac{d\nu}{dr} = \frac{1}{p + \epsilon} \frac{dp}{dr}$

where $\frac{dp}{dr}$ is obtained using the TOV solver which was described in my first research journala, p is the pressure, and $\epsilon$ is the energy density. I integrate this using scipy.integrate.solve_ivp to obtain $\nu(r)$. From here I compute $g_{00}$ as follows

$g_{00}=e^{2\nu}$

finally the schwarzschild solution 

S = $1 - \frac{2GM}{R}$

Although at the moment my code is rather chaotic as I am in the process of debugging, here is the code I have.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.integrate as integ
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Define constants
# Constants
m_b = 1 # GeV, boson mass
l = 1 # Coupling strength
e_0 = l * m_b**4 # GeV^4, initial energy density
G = 1.19e19 # GeV, Gravitational constant
c = 1 # Speed of light
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Define equations
def TOV(r, y):
    """
    Here I have removed the c from the equations as it is 1.
    """
    p, m = y
    if p <= 0: # Stop integration if pressure drops below zero
        return [0, 0]

    e = e_0*(p/e_0)**0.5 + 3*p # EOS
    dp_dr = (-G/(r**2))*(e + p)*(4*np.pi*(r**3)*p)*(1 - (2*G*m)/r)**-1
    dm_dr = 4*np.pi*(r**2)*e
    return [dp_dr, dm_dr]

def Glendenning(r, y):
    """
    system of differential equations to pass to the scipy.integrate.solve_ivp
    """
    p, m, v = y # Extract p, m, and v from y
    e = e_0 * (p / e_0)**0.5 + 3 * p # EOS
    dp_dr, dm_dr = TOV(r, [p, m]) # Extract derivatives from TOV

    dv_dr = - (1 / (p + e)) * dp_dr

    return [dp_dr, dm_dr, dv_dr] # Return all three derivatives
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Integrate
r0 = 1e7 # Initial radius to avoid division by zero
v0 = 0 # initial nu value 
r_max = 1e11 # Maximum radius
p0 = 1e-8 # intial pressure
m0 = 0 # initial mass
y0 = [p0, m0, v0] # vector containing the initial conditions to pass to scipy.integrate.solve_ivp
rspan = [r0, r_max] # bounds of integration to pass to scipy.integrate.solve_ivp
reval = np.linspace(r0, r_max, 100) # r values to evaluate the integral at in scipy.integrate.solve_ivp

solution = integ.solve_ivp(Glendenning, rspan, y0, t_eval = reval) # solve the system of differential equations

# Extract the arrays for radius (r), pressure(p), mass (m), and nu (v) from the solution from scipy.integrate.solve_ivp
r = solution.t
p = solution.y[0]
m = solution.y[1]
v = solution.y[2]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# compute the metric function
g00 = np.exp(2*v)

# Test values to compute the Schwarzschild radius over
M = 2e57 # 2 solar masses? correct this later
R = r_max # maximum radius of the dark star 

r_space = np.linspace(r[-1], 2*r_max, 100) # list of r values outside the dark star to compute the Schwarzschild solution at.
schwarzschild = 1 - ((2*G*M)/(r_space)) # compute the Schwarzschild solution

C = schwarzschild[0] / g00[-1] # coefficient to make the metric function g00 match the Schwarzschild at radius r = R

g00_corrected = C*g00 # apply the coefficient to the metric function.
"""
note that this g00 corrected worked in matching up the values at r=R which was observed graphically, however I am unable to supply this
graph as my code no longer runs.
"""
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Potential energy and redshift factor
m = 1 # test mass
GM_r = 0.5*(1 - g00) # intermediate step for calculations

U = m*GM_r # potential energy (for when the metric function is correctly solved for)

Z = (1/g00) - 1 # redshift factor (for when the metric function is correctly solved for)
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Plot 
plt.plot(r , g00, label=r"$g_{00}$")
plt.title(r"$g_{00} \ Vs. \ radius$", fontsize=14)
plt.xlabel(r"radius $(GeV^-1)$", fontsize=14)
plt.ylabel(r"$g_{00} (dimensionless)$", fontsize=14)
plt.grid()
plt.legend(fontsize=14)
plt.show()

plt.plot(r, U, label="U(r)")
plt.xlabel(r"radius $(GeV^-1)$", fontsize=14)
plt.ylabel("U(r) GeV", fontsize=14)
plt.title("Potential Energy vs Radius")
plt.legend(fontsize=14)
plt.grid()
plt.show()

plt.plot(r, Z, label="Z(r)")
plt.xlabel(r"radius $(GeV^{-1})$", fontsize=14)
plt.ylabel("Z(r)", fontsize=14)
plt.title("Z factor vs Radius")
plt.legend(fontsize=14)
plt.grid()
plt.show()

plt.plot(r, g00, label=r"$g_{00}$")
plt.plot(r_space, schwarzschild, label="schwarzschild solution")
plt.title(r"$g_{00} \ Vs. \ radius$", fontsize=14)
plt.xlabel(r"radius $(GeV^-1)$", fontsize=14)
plt.ylabel(r"$g_{00} (dimensionless)$", fontsize=14)
plt.xlim(r0, r[-1])
plt.grid()
plt.legend(fontsize=14)
plt.show()
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

C:\Users\socce\AppData\Local\Temp\ipykernel_12240\3051892294.py:36: RuntimeWarning: invalid value encountered in scalar power
  e = e_0 * (p / e_0)**0.5 + 3 * p # EOS


IndexError: list index out of range

# 3: So what? (What does it mean?)
## Describe your results

Although I do not believe this has been the most successful couple of weeks since my last journal, as I did not complete the metric function, redshift factor, and potential energy calculations, I think I am close to the solution. I have some ideas on how to fix the code, which I will describe in the next section. Despite the setbacks, I am still pleased with my progress because I now have a much clearer understanding of the physical processes involved. One of the main points of confusion was whether the Schwarzschild solution applies inside the star, and I now realize that it does not. This is precisely why we need the metric function to describe the interior solution properly. Additionally, I have made solid progress on this goal, and once I correct the code, I should be able to move forward. At that point, I can combine my results with those of my peers to finalize our project. My peers are working on determining the temperature, accretion rate, and mass-radius curves for the two-fluid case, which includes both ordinary matter and dark matter. The metric function I am developing is a necessary component for one of my peers' calculations, and the redshift factor will allow us to properly account for gravitational redshift. These calculations are essential because, once completed and combined, they will enable us to determine the wavelength a dark star would emit due to accretion. While this has been a challenging process, I feel that I am making steady progress toward the final results.

# 4. Now what? (What's next?)
## Plan for the next week

This week I did not achieve my intended goal of completing the metric function, potential energy, and z factor calculations. I think from here I need to take a new, more rigorous approach and put in proper values for the mass and the radii and be more consistent in these numbers. A few other things I intend to implement is to remove the specified points to evaluate the radius, allowing solve_ivp to use adaptive stepsize. Additionally, I will implement a terminate condition for one then pressure equals zero (defining the edge of the dark star) and switch from RK45 to DOP853 method. Additionally next week me and the other students in my group plan to meet this weekend to begin discussing how we are going to do our presentation.

# 5. Bibliography


[1] N. K. Glendenning, Compact Stars: Nuclear Physics, Particle Physics, and General Relativity. Springer, 1997.